In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
import tensorflow
print(tensorflow.__version__)

2.1.0


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8851981156477701508
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3063362355
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5390477054831507765
physical_device_desc: "device: 0, name: Quadro T1000, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
tensorflow.executing_eagerly()

True

### Testing keras tuner on IRIS dataset

In [26]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [28]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a bayesian tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [29]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3, #5,
    executions_per_trial=2, #3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_5_epochs',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

## Efecto de meter más epochs

In [30]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test) #,
             #batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/10
 32/100 [========>.....................] - ETA: 0s - loss: 1.0848 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.0285 - accuracy: 0.3500 - val_loss: 0.9286 - val_accuracy: 0.5600
Epoch 2/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.9295 - accuracy: 0.43100/100 [==============================] - 0s 748us/sample - loss: 0.9122 - accuracy: 0.5700 - val_loss: 0.8453 - val_accuracy: 0.7000
Epoch 3/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.8661 - accuracy: 0.62100/100 [==============================] - 0s 239us/sample - loss: 0.8370 - accuracy: 0.6800 - val_loss: 0.7643 - val_accuracy: 0.7000
Epoch 4/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.8073 - accuracy: 0.59100/100 [==============================] - 0s 279us/sample - loss: 0.7750 - accuracy: 0.6500 - val_loss: 0.7011 - val_accuracy: 0.7000
Epoch 5/10
 32/100 [========>................

Train on 100 samples, validate on 50 samples
Epoch 1/10
 32/100 [========>.....................] - ETA: 0s - loss: 1.1120 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.1183 - accuracy: 0.3400 - val_loss: 1.0473 - val_accuracy: 0.3200
Epoch 2/10
 32/100 [========>.....................] - ETA: 0s - loss: 1.0502 - accuracy: 0.31100/100 [==============================] - 0s 189us/sample - loss: 1.0117 - accuracy: 0.3400 - val_loss: 0.9707 - val_accuracy: 0.3200
Epoch 3/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.9661 - accuracy: 0.34100/100 [==============================] - 0s 778us/sample - loss: 0.9398 - accuracy: 0.3600 - val_loss: 0.8772 - val_accuracy: 0.7000
Epoch 4/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.9118 - accuracy: 0.46100/100 [==============================] - 0s 788us/sample - loss: 0.8635 - accuracy: 0.6500 - val_loss: 0.7948 - val_accuracy: 0.8600
Epoch 5/10
 32/100 [========>................

Train on 100 samples, validate on 50 samples
Epoch 1/10
 32/100 [========>.....................] - ETA: 0s - loss: 1.8012 - accuracy: 0.43100/100 [==============================] - 0s 4ms/sample - loss: 1.6189 - accuracy: 0.3500 - val_loss: 0.9679 - val_accuracy: 0.5600
Epoch 2/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.9487 - accuracy: 0.53100/100 [==============================] - 0s 678us/sample - loss: 0.9852 - accuracy: 0.5700 - val_loss: 0.8884 - val_accuracy: 0.7000
Epoch 3/10
 32/100 [========>.....................] - ETA: 0s - loss: 1.0536 - accuracy: 0.56100/100 [==============================] - 0s 199us/sample - loss: 0.9223 - accuracy: 0.6500 - val_loss: 0.7563 - val_accuracy: 0.7000
Epoch 4/10
 32/100 [========>.....................] - ETA: 0s - loss: 0.8335 - accuracy: 0.65100/100 [==============================] - 0s 189us/sample - loss: 0.7759 - accuracy: 0.6500 - val_loss: 0.6386 - val_accuracy: 0.7000
Epoch 5/10
 32/100 [========>................

INFO:tensorflow:Oracle triggered exit


In [32]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3, #5,
    executions_per_trial=2, #3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_5_epochs',
    overwrite=True)

bayesian_opt_tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_test, y_test) #,
             #batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.1442 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.1236 - accuracy: 0.3500 - val_loss: 1.1212 - val_accuracy: 0.3000
Epoch 2/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.1173 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 1.0998 - accuracy: 0.3500 - val_loss: 1.0989 - val_accuracy: 0.3000
Epoch 3/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.0901 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 1.0793 - accuracy: 0.3500 - val_loss: 1.0796 - val_accuracy: 0.3000
Epoch 4/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.0486 - accuracy: 0.37100/100 [==============================] - 0s 189us/sample - loss: 1.0609 - accuracy: 0.3500 - val_loss: 1.0616 - val_accuracy: 0.3000
Epoch 5/30
 32/100 [========>................

Train on 100 samples, validate on 50 samples
Epoch 1/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.1107 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.1332 - accuracy: 0.5100 - val_loss: 0.9457 - val_accuracy: 0.6800
Epoch 2/30
 32/100 [========>.....................] - ETA: 0s - loss: 0.9641 - accuracy: 0.65100/100 [==============================] - 0s 758us/sample - loss: 0.9411 - accuracy: 0.6600 - val_loss: 0.8936 - val_accuracy: 0.7000
Epoch 3/30
 32/100 [========>.....................] - ETA: 0s - loss: 0.8958 - accuracy: 0.71100/100 [==============================] - 0s 209us/sample - loss: 0.8904 - accuracy: 0.6400 - val_loss: 0.8271 - val_accuracy: 0.7000
Epoch 4/30
 32/100 [========>.....................] - ETA: 0s - loss: 0.8659 - accuracy: 0.56100/100 [==============================] - 0s 788us/sample - loss: 0.8229 - accuracy: 0.6500 - val_loss: 0.7635 - val_accuracy: 0.7200
Epoch 5/30
 32/100 [========>................

Train on 100 samples, validate on 50 samples
Epoch 1/30
 32/100 [========>.....................] - ETA: 0s - loss: 2.7273 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 2.3185 - accuracy: 0.3100 - val_loss: 1.2611 - val_accuracy: 0.3800
Epoch 2/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.3768 - accuracy: 0.28100/100 [==============================] - 0s 209us/sample - loss: 1.1895 - accuracy: 0.3000 - val_loss: 1.0231 - val_accuracy: 0.3600
Epoch 3/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.0035 - accuracy: 0.28100/100 [==============================] - 0s 758us/sample - loss: 1.0393 - accuracy: 0.4200 - val_loss: 1.1230 - val_accuracy: 0.5200
Epoch 4/30
 32/100 [========>.....................] - ETA: 0s - loss: 1.2169 - accuracy: 0.50100/100 [==============================] - 0s 229us/sample - loss: 1.0474 - accuracy: 0.5500 - val_loss: 1.0893 - val_accuracy: 0.3200
Epoch 5/30
 32/100 [========>................

INFO:tensorflow:Oracle triggered exit


### Voy a probar con otro batch_size  

In [33]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_10_epochs',
    overwrite=True)

training_history = bayesian_opt_tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.0747 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.0517 - accuracy: 0.3200 - val_loss: 1.0072 - val_accuracy: 0.3200
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.0122 - accuracy: 0.28100/100 [==============================] - 0s 668us/sample - loss: 0.9764 - accuracy: 0.3400 - val_loss: 0.9282 - val_accuracy: 0.7000
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.8864 - accuracy: 0.80100/100 [==============================] - 0s 130us/sample - loss: 0.9319 - accuracy: 0.6500 - val_loss: 0.8689 - val_accuracy: 0.7000
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.8634 - accuracy: 0.70100/100 [==============================] - 0s 130us/sample - loss: 0.8741 - accuracy: 0.6500 - val_loss: 0.8177 - val_accuracy: 0.7000
Epoch 5/10
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.4071 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.2883 - accuracy: 0.3400 - val_loss: 1.3867 - val_accuracy: 0.3200
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.1365 - accuracy: 0.38100/100 [==============================] - 0s 120us/sample - loss: 1.2785 - accuracy: 0.3400 - val_loss: 1.3773 - val_accuracy: 0.3200
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.3762 - accuracy: 0.30100/100 [==============================] - 0s 120us/sample - loss: 1.2695 - accuracy: 0.3400 - val_loss: 1.3679 - val_accuracy: 0.3200
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.2797 - accuracy: 0.30100/100 [==============================] - 0s 140us/sample - loss: 1.2607 - accuracy: 0.3400 - val_loss: 1.3588 - val_accuracy: 0.3200
Epoch 5/10
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.1921 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.3603 - accuracy: 0.3900 - val_loss: 0.8538 - val_accuracy: 0.7000
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.8515 - accuracy: 0.70100/100 [==============================] - 0s 130us/sample - loss: 0.8624 - accuracy: 0.6900 - val_loss: 0.9065 - val_accuracy: 0.6800
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.8344 - accuracy: 0.72100/100 [==============================] - 0s 140us/sample - loss: 0.7669 - accuracy: 0.6600 - val_loss: 0.5281 - val_accuracy: 0.7000
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.6198 - accuracy: 0.62100/100 [==============================] - 0s 120us/sample - loss: 0.6329 - accuracy: 0.6500 - val_loss: 0.5704 - val_accuracy: 0.7000
Epoch 5/10
 50/100 [==============>..........

INFO:tensorflow:Oracle triggered exit


In [34]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3, #5,
    executions_per_trial=2, #3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_5_epochs',
    overwrite=True)

bayesian_opt_tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2))


Train on 100 samples, validate on 50 samples
Epoch 1/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.2249 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.1958 - accuracy: 0.3300 - val_loss: 0.9999 - val_accuracy: 0.6000
Epoch 2/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.9641 - accuracy: 0.62100/100 [==============================] - 0s 659us/sample - loss: 1.0024 - accuracy: 0.6100 - val_loss: 0.8825 - val_accuracy: 0.6800
Epoch 3/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.9658 - accuracy: 0.58100/100 [==============================] - 0s 678us/sample - loss: 0.9039 - accuracy: 0.7100 - val_loss: 0.7776 - val_accuracy: 0.8400
Epoch 4/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.7941 - accuracy: 0.74100/100 [==============================] - 0s 120us/sample - loss: 0.7693 - accuracy: 0.8300 - val_loss: 0.7167 - val_accuracy: 0.8200
Epoch 5/30
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.1442 - accuracy: 0.0000e+100/100 [==============================] - 0s 4ms/sample - loss: 1.1069 - accuracy: 0.1500 - val_loss: 1.0617 - val_accuracy: 0.3000
Epoch 2/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.9691 - accuracy: 0.38100/100 [==============================] - 0s 659us/sample - loss: 0.9814 - accuracy: 0.3500 - val_loss: 0.8799 - val_accuracy: 0.6800
Epoch 3/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.8722 - accuracy: 0.60100/100 [==============================] - 0s 648us/sample - loss: 0.8472 - accuracy: 0.6300 - val_loss: 0.7536 - val_accuracy: 0.7000
Epoch 4/30
 50/100 [==============>...............] - ETA: 0s - loss: 0.7835 - accuracy: 0.68100/100 [==============================] - 0s 130us/sample - loss: 0.7778 - accuracy: 0.6500 - val_loss: 0.6717 - val_accuracy: 0.7000
Epoch 5/30
 50/100 [==============>......

Train on 100 samples, validate on 50 samples
Epoch 1/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.0757 - accuracy: 0.50100/100 [==============================] - 0s 4ms/sample - loss: 1.0692 - accuracy: 0.5700 - val_loss: 1.0452 - val_accuracy: 0.6600
Epoch 2/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.0549 - accuracy: 0.64100/100 [==============================] - 0s 140us/sample - loss: 1.0575 - accuracy: 0.5800 - val_loss: 1.0366 - val_accuracy: 0.5600
Epoch 3/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.0545 - accuracy: 0.54100/100 [==============================] - 0s 140us/sample - loss: 1.0472 - accuracy: 0.5000 - val_loss: 1.0286 - val_accuracy: 0.4600
Epoch 4/30
 50/100 [==============>...............] - ETA: 0s - loss: 1.0270 - accuracy: 0.44100/100 [==============================] - 0s 130us/sample - loss: 1.0366 - accuracy: 0.4500 - val_loss: 1.0209 - val_accuracy: 0.4200
Epoch 5/30
 50/100 [==============>..........

INFO:tensorflow:Oracle triggered exit


## En principio sí mejora el score elñ hecho de meter más epochs, pero probar con mayor número de ejemplos y con dataset más complicado 

## Efecto de reducir batch_size en la evolución del entrenamiento a lo largo de los epochs? Pensar en estudio de combinación epochs_number/batch_size

In [24]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_30_epochs_small_batch',
    overwrite=True)

training_history = bayesian_opt_tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/6)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/30
 16/100 [===>..........................] - ETA: 1s - loss: 1.2179 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.1682 - accuracy: 0.3100 - val_loss: 1.1210 - val_accuracy: 0.3800
Epoch 2/30
 16/100 [===>..........................] - ETA: 0s - loss: 1.1242 - accuracy: 0.31100/100 [==============================] - 0s 858us/sample - loss: 1.1236 - accuracy: 0.2900 - val_loss: 1.0922 - val_accuracy: 0.4800
Epoch 3/30
 16/100 [===>..........................] - ETA: 0s - loss: 1.1076 - accuracy: 0.37100/100 [==============================] - 0s 309us/sample - loss: 1.0931 - accuracy: 0.4300 - val_loss: 1.0688 - val_accuracy: 0.4200
Epoch 4/30
 16/100 [===>..........................] - ETA: 0s - loss: 1.0458 - accuracy: 0.56100/100 [==============================] - 0s 249us/sample - loss: 1.0621 - accuracy: 0.3500 - val_loss: 1.0485 - val_accuracy: 0.4000
Epoch 5/30
 16/100 [===>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/30
 16/100 [===>..........................] - ETA: 1s - loss: 1.1453 - accuracy: 0.12100/100 [==============================] - 0s 4ms/sample - loss: 0.9343 - accuracy: 0.4900 - val_loss: 0.5495 - val_accuracy: 0.6600
Epoch 2/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.6321 - accuracy: 0.50100/100 [==============================] - 0s 848us/sample - loss: 0.6138 - accuracy: 0.6200 - val_loss: 0.6659 - val_accuracy: 0.6800
Epoch 3/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.7958 - accuracy: 0.62100/100 [==============================] - 0s 878us/sample - loss: 0.6081 - accuracy: 0.7100 - val_loss: 0.4215 - val_accuracy: 0.7000
Epoch 4/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.6191 - accuracy: 0.56100/100 [==============================] - 0s 888us/sample - loss: 0.4082 - accuracy: 0.7600 - val_loss: 0.3656 - val_accuracy: 0.7400
Epoch 5/30
 16/100 [===>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/30
 16/100 [===>..........................] - ETA: 1s - loss: 1.4793 - accuracy: 0.18100/100 [==============================] - 0s 4ms/sample - loss: 0.9723 - accuracy: 0.4200 - val_loss: 0.8105 - val_accuracy: 0.4600
Epoch 2/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.8114 - accuracy: 0.50100/100 [==============================] - 0s 878us/sample - loss: 0.7320 - accuracy: 0.5500 - val_loss: 0.5646 - val_accuracy: 0.9200
Epoch 3/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.5724 - accuracy: 0.87100/100 [==============================] - 0s 808us/sample - loss: 0.5925 - accuracy: 0.7200 - val_loss: 0.4509 - val_accuracy: 0.9600
Epoch 4/30
 16/100 [===>..........................] - ETA: 0s - loss: 0.5486 - accuracy: 0.87100/100 [==============================] - 0s 299us/sample - loss: 0.5308 - accuracy: 0.8300 - val_loss: 0.3925 - val_accuracy: 0.9200
Epoch 5/30
 16/100 [===>.....................

INFO:tensorflow:Oracle triggered exit


### as expected, cuantos más epochs mejor entrenamiento (sin entrar en posible sobreajuste aún); para combinaciones de hiperparams. muy buenas parece converger muy rápido, y no podemos ver si los accuracy scores tienen una evolución "errática", pero en los primeros trials parece que sí es errático... a ver la estabilidad de los accuracy values al aumentar el batch_size

In [25]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_30_epochs_big_batch',
    overwrite=True)

training_history = bayesian_opt_tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/1.2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.3472 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.4384 - accuracy: 0.3000 - val_loss: 0.9096 - val_accuracy: 0.3000
Epoch 2/30
 83/100 [=======================>......] - ETA: 0s - loss: 0.8926 - accuracy: 0.37100/100 [==============================] - 0s 698us/sample - loss: 0.8552 - accuracy: 0.4400 - val_loss: 0.9086 - val_accuracy: 0.7000
Epoch 3/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.0840 - accuracy: 0.61100/100 [==============================] - 0s 150us/sample - loss: 0.9913 - accuracy: 0.6500 - val_loss: 0.7318 - val_accuracy: 0.7000
Epoch 4/30
 83/100 [=======================>......] - ETA: 0s - loss: 0.8483 - accuracy: 0.63100/100 [==============================] - 0s 738us/sample - loss: 0.7965 - accuracy: 0.6500 - val_loss: 0.4661 - val_accuracy: 0.9600
Epoch 5/30
 83/100 [=======================>.

Train on 100 samples, validate on 50 samples
Epoch 1/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.6280 - accuracy: 0.37100/100 [==============================] - 0s 4ms/sample - loss: 1.6371 - accuracy: 0.3400 - val_loss: 1.0932 - val_accuracy: 0.3200
Epoch 2/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.1153 - accuracy: 0.33100/100 [==============================] - 0s 140us/sample - loss: 1.1001 - accuracy: 0.3900 - val_loss: 1.0558 - val_accuracy: 0.3000
Epoch 3/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.0719 - accuracy: 0.32100/100 [==============================] - 0s 140us/sample - loss: 1.0515 - accuracy: 0.3500 - val_loss: 1.0898 - val_accuracy: 0.3000
Epoch 4/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.0678 - accuracy: 0.34100/100 [==============================] - 0s 678us/sample - loss: 1.0679 - accuracy: 0.3500 - val_loss: 0.9304 - val_accuracy: 0.3400
Epoch 5/30
 83/100 [=======================>.

Train on 100 samples, validate on 50 samples
Epoch 1/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.1713 - accuracy: 0.33100/100 [==============================] - 0s 4ms/sample - loss: 1.1564 - accuracy: 0.3500 - val_loss: 1.1400 - val_accuracy: 0.3000
Epoch 2/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.1281 - accuracy: 0.36100/100 [==============================] - 0s 150us/sample - loss: 1.1385 - accuracy: 0.3500 - val_loss: 1.1243 - val_accuracy: 0.3000
Epoch 3/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.1336 - accuracy: 0.32100/100 [==============================] - 0s 150us/sample - loss: 1.1235 - accuracy: 0.3500 - val_loss: 1.1097 - val_accuracy: 0.3000
Epoch 4/30
 83/100 [=======================>......] - ETA: 0s - loss: 1.1180 - accuracy: 0.32100/100 [==============================] - 0s 140us/sample - loss: 1.1084 - accuracy: 0.3500 - val_loss: 1.0964 - val_accuracy: 0.3000
Epoch 5/30
 83/100 [=======================>.

INFO:tensorflow:Oracle triggered exit


## Estudiar con más experimentos el efecto de disminuir el batch_size para ver si:
* afecta al validation score final suponiendo un número suficiente de epochs
* si afecta a la estabilidad del accuracy score a lo largo de los training epochs
* a la velocidad del entrenamiento

### Vi que el batch_size por defecto parece ser 1/3 del training set

In [13]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.3250 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.1663 - accuracy: 0.3200 - val_loss: 0.9806 - val_accuracy: 0.3000
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.9257 - accuracy: 0.40100/100 [==============================] - 0s 638us/sample - loss: 0.9037 - accuracy: 0.5000 - val_loss: 0.6921 - val_accuracy: 0.8800
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.7103 - accuracy: 0.94100/100 [==============================] - 0s 199us/sample - loss: 0.7349 - accuracy: 0.7800 - val_loss: 0.6412 - val_accuracy: 0.7000
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.6702 - accuracy: 0.68100/100 [==============================] - 0s 269us/sample - loss: 0.6782 - accuracy: 0.6500 - val_loss: 0.5194 - val_accuracy: 0.7000
Epoch 5/10
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.1293 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.4243 - accuracy: 0.3200 - val_loss: 0.8715 - val_accuracy: 0.7000
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.9442 - accuracy: 0.66100/100 [==============================] - 0s 130us/sample - loss: 0.8163 - accuracy: 0.6900 - val_loss: 0.8575 - val_accuracy: 0.6800
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.8337 - accuracy: 0.70100/100 [==============================] - 0s 658us/sample - loss: 0.8028 - accuracy: 0.6600 - val_loss: 0.5008 - val_accuracy: 1.0000
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 0.5219 - accuracy: 0.96100/100 [==============================] - 0s 140us/sample - loss: 0.5193 - accuracy: 0.8200 - val_loss: 0.5096 - val_accuracy: 0.7000
Epoch 5/10
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.3832 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.2954 - accuracy: 0.3500 - val_loss: 1.3487 - val_accuracy: 0.3000
Epoch 2/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.3246 - accuracy: 0.32100/100 [==============================] - 0s 120us/sample - loss: 1.2629 - accuracy: 0.3500 - val_loss: 1.3158 - val_accuracy: 0.3000
Epoch 3/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.1991 - accuracy: 0.38100/100 [==============================] - 0s 130us/sample - loss: 1.2331 - accuracy: 0.3500 - val_loss: 1.2846 - val_accuracy: 0.3000
Epoch 4/10
 50/100 [==============>...............] - ETA: 0s - loss: 1.2299 - accuracy: 0.34100/100 [==============================] - 0s 120us/sample - loss: 1.2063 - accuracy: 0.3500 - val_loss: 1.2548 - val_accuracy: 0.3000
Epoch 5/10
 50/100 [==============>..........

INFO:tensorflow:Oracle triggered exit


In [7]:
bayesian_opt_tuner.results_summary()

## Ahora con más trials, quiero comprobar si con suficientes hiperparams. combinations, se va dirigiendo a mejores scores

In [8]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=15,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_5_epochs',
    overwrite=True)


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1390 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.1040 - accuracy: 0.3500 - val_loss: 1.0407 - val_accuracy: 0.5600
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.0296 - accuracy: 0.60100/100 [==============================] - 0s 150us/sample - loss: 1.0150 - accuracy: 0.4600 - val_loss: 0.9870 - val_accuracy: 0.3200
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9730 - accuracy: 0.34100/100 [==============================] - 0s 160us/sample - loss: 0.9660 - accuracy: 0.3400 - val_loss: 0.9424 - val_accuracy: 0.3200
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9189 - accuracy: 0.36100/100 [==============================] - 0s 140us/sample - loss: 0.9228 - accuracy: 0.3400 - val_loss: 0.8891 - val_accuracy: 0.5000
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1688 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.4589 - accuracy: 0.3100 - val_loss: 0.8897 - val_accuracy: 0.7000
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9431 - accuracy: 0.58100/100 [==============================] - 0s 119us/sample - loss: 0.9051 - accuracy: 0.6300 - val_loss: 0.9504 - val_accuracy: 0.6800
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9316 - accuracy: 0.70100/100 [==============================] - 0s 120us/sample - loss: 0.9465 - accuracy: 0.6500 - val_loss: 0.6046 - val_accuracy: 0.6800
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.6393 - accuracy: 0.72100/100 [==============================] - 0s 130us/sample - loss: 0.5738 - accuracy: 0.7700 - val_loss: 0.5346 - val_accuracy: 0.7000
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.3115 - accuracy: 0.26100/100 [==============================] - 0s 4ms/sample - loss: 1.2490 - accuracy: 0.2600 - val_loss: 1.1047 - val_accuracy: 0.3200
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.0860 - accuracy: 0.32100/100 [==============================] - 0s 130us/sample - loss: 1.0355 - accuracy: 0.3400 - val_loss: 0.9402 - val_accuracy: 0.3200
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9096 - accuracy: 0.38100/100 [==============================] - 0s 668us/sample - loss: 0.8866 - accuracy: 0.3900 - val_loss: 0.8095 - val_accuracy: 0.8400
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.8101 - accuracy: 0.78100/100 [==============================] - 0s 659us/sample - loss: 0.7828 - accuracy: 0.8100 - val_loss: 0.7018 - val_accuracy: 0.8600
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.5418 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.3826 - accuracy: 0.3300 - val_loss: 1.2433 - val_accuracy: 0.3200
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1080 - accuracy: 0.40100/100 [==============================] - 0s 638us/sample - loss: 1.0144 - accuracy: 0.3400 - val_loss: 0.6361 - val_accuracy: 0.7000
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.6592 - accuracy: 0.66100/100 [==============================] - 0s 130us/sample - loss: 0.7109 - accuracy: 0.6500 - val_loss: 0.6053 - val_accuracy: 0.6800
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.6896 - accuracy: 0.78100/100 [==============================] - 0s 120us/sample - loss: 0.6486 - accuracy: 0.6900 - val_loss: 0.4692 - val_accuracy: 0.7000
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1888 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.4132 - accuracy: 0.2900 - val_loss: 1.3127 - val_accuracy: 0.3200
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1207 - accuracy: 0.40100/100 [==============================] - 0s 649us/sample - loss: 1.1345 - accuracy: 0.3500 - val_loss: 0.6451 - val_accuracy: 0.7000
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.7085 - accuracy: 0.64100/100 [==============================] - 0s 130us/sample - loss: 0.7363 - accuracy: 0.5800 - val_loss: 0.7926 - val_accuracy: 0.6600
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.8230 - accuracy: 0.64100/100 [==============================] - 0s 638us/sample - loss: 0.7753 - accuracy: 0.6500 - val_loss: 0.4904 - val_accuracy: 0.8000
Epoch 5/5
 50/100 [==============>...............

...

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1488 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.4549 - accuracy: 0.3300 - val_loss: 1.2921 - val_accuracy: 0.3800
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.5250 - accuracy: 0.28100/100 [==============================] - 0s 678us/sample - loss: 1.1805 - accuracy: 0.4300 - val_loss: 0.8443 - val_accuracy: 0.4200
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.8156 - accuracy: 0.46100/100 [==============================] - 0s 110us/sample - loss: 0.8459 - accuracy: 0.3900 - val_loss: 0.7713 - val_accuracy: 0.3800
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.7458 - accuracy: 0.40100/100 [==============================] - 0s 708us/sample - loss: 0.7550 - accuracy: 0.5000 - val_loss: 0.6156 - val_accuracy: 0.7000
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.0274 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.5101 - accuracy: 0.4300 - val_loss: 0.8738 - val_accuracy: 0.7000
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.9259 - accuracy: 0.68100/100 [==============================] - 0s 139us/sample - loss: 0.8297 - accuracy: 0.7000 - val_loss: 0.9964 - val_accuracy: 0.5600
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.0964 - accuracy: 0.52100/100 [==============================] - 0s 150us/sample - loss: 0.8798 - accuracy: 0.5900 - val_loss: 0.5480 - val_accuracy: 0.7000
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.5371 - accuracy: 0.74100/100 [==============================] - 0s 130us/sample - loss: 0.6822 - accuracy: 0.6500 - val_loss: 0.5522 - val_accuracy: 0.7000
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1794 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.2917 - accuracy: 0.3300 - val_loss: 1.2726 - val_accuracy: 0.3200
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.3137 - accuracy: 0.32100/100 [==============================] - 0s 698us/sample - loss: 1.0258 - accuracy: 0.6400 - val_loss: 0.6993 - val_accuracy: 0.6800
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.7404 - accuracy: 0.68100/100 [==============================] - 0s 668us/sample - loss: 0.7379 - accuracy: 0.5700 - val_loss: 0.5896 - val_accuracy: 0.7000
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.6286 - accuracy: 0.70100/100 [==============================] - 0s 668us/sample - loss: 0.6511 - accuracy: 0.6500 - val_loss: 0.4418 - val_accuracy: 0.8400
Epoch 5/5
 50/100 [==============>...............

Train on 100 samples, validate on 50 samples
Epoch 1/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1970 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.4236 - accuracy: 0.2900 - val_loss: 1.2993 - val_accuracy: 0.3200
Epoch 2/5
 50/100 [==============>...............] - ETA: 0s - loss: 1.1469 - accuracy: 0.40100/100 [==============================] - 0s 679us/sample - loss: 1.1230 - accuracy: 0.3900 - val_loss: 0.6228 - val_accuracy: 0.7000
Epoch 3/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.6631 - accuracy: 0.66100/100 [==============================] - 0s 140us/sample - loss: 0.7607 - accuracy: 0.5200 - val_loss: 0.7410 - val_accuracy: 0.6000
Epoch 4/5
 50/100 [==============>...............] - ETA: 0s - loss: 0.7576 - accuracy: 0.66100/100 [==============================] - 0s 688us/sample - loss: 0.7242 - accuracy: 0.6400 - val_loss: 0.4616 - val_accuracy: 0.9600
Epoch 5/5
 50/100 [==============>...............

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()

### Ahora con más epochs y 15 trials

In [11]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=15,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_15_epochs',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             epochs=20,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1515 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.2808 - accuracy: 0.3700 - val_loss: 0.7785 - val_accuracy: 0.7000
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.7932 - accuracy: 0.58100/100 [==============================] - 0s 130us/sample - loss: 0.8642 - accuracy: 0.6400 - val_loss: 0.8563 - val_accuracy: 0.6800
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.9535 - accuracy: 0.64100/100 [==============================] - 0s 150us/sample - loss: 0.8202 - accuracy: 0.5700 - val_loss: 0.5526 - val_accuracy: 0.7000
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.6218 - accuracy: 0.64100/100 [==============================] - 0s 120us/sample - loss: 0.6268 - accuracy: 0.6500 - val_loss: 0.5400 - val_accuracy: 0.7000
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1628 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.2006 - accuracy: 0.3400 - val_loss: 1.2411 - val_accuracy: 0.3200
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.3070 - accuracy: 0.26100/100 [==============================] - 0s 130us/sample - loss: 1.1838 - accuracy: 0.3400 - val_loss: 1.2228 - val_accuracy: 0.3200
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1616 - accuracy: 0.34100/100 [==============================] - 0s 140us/sample - loss: 1.1661 - accuracy: 0.3400 - val_loss: 1.2053 - val_accuracy: 0.3200
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.3174 - accuracy: 0.20100/100 [==============================] - 0s 130us/sample - loss: 1.1517 - accuracy: 0.3400 - val_loss: 1.1885 - val_accuracy: 0.3200
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.5898 - accuracy: 0.26100/100 [==============================] - 0s 4ms/sample - loss: 1.3769 - accuracy: 0.3300 - val_loss: 1.1130 - val_accuracy: 0.3200
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.0537 - accuracy: 0.46100/100 [==============================] - 0s 120us/sample - loss: 1.0468 - accuracy: 0.4200 - val_loss: 1.0163 - val_accuracy: 0.3200
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.0146 - accuracy: 0.32100/100 [==============================] - 0s 140us/sample - loss: 0.9716 - accuracy: 0.3600 - val_loss: 0.9194 - val_accuracy: 0.3000
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.9065 - accuracy: 0.38100/100 [==============================] - 0s 658us/sample - loss: 0.8903 - accuracy: 0.3700 - val_loss: 0.7873 - val_accuracy: 0.7000
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.2650 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.3745 - accuracy: 0.2600 - val_loss: 0.9611 - val_accuracy: 0.6800
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.0164 - accuracy: 0.60100/100 [==============================] - 0s 778us/sample - loss: 0.9253 - accuracy: 0.5900 - val_loss: 0.7593 - val_accuracy: 0.7000
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.9093 - accuracy: 0.58100/100 [==============================] - 0s 189us/sample - loss: 0.7946 - accuracy: 0.6500 - val_loss: 0.6076 - val_accuracy: 0.7000
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.6102 - accuracy: 0.70100/100 [==============================] - 0s 658us/sample - loss: 0.6108 - accuracy: 0.6500 - val_loss: 0.4990 - val_accuracy: 0.8600
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1901 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.4028 - accuracy: 0.2900 - val_loss: 1.6563 - val_accuracy: 0.3000
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.5195 - accuracy: 0.38100/100 [==============================] - 0s 758us/sample - loss: 1.1750 - accuracy: 0.5100 - val_loss: 0.7473 - val_accuracy: 0.7000
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.8200 - accuracy: 0.64100/100 [==============================] - 0s 140us/sample - loss: 0.8326 - accuracy: 0.6500 - val_loss: 0.7340 - val_accuracy: 0.6000
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.7728 - accuracy: 0.62100/100 [==============================] - 0s 758us/sample - loss: 0.7231 - accuracy: 0.6100 - val_loss: 0.4645 - val_accuracy: 0.8200
Epoch 5/20
 50/100 [==============>..........

...

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1849 - accuracy: 0.62100/100 [==============================] - 0s 4ms/sample - loss: 1.4172 - accuracy: 0.4700 - val_loss: 0.8668 - val_accuracy: 0.7000
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.9276 - accuracy: 0.62100/100 [==============================] - 0s 160us/sample - loss: 0.8482 - accuracy: 0.6600 - val_loss: 0.9320 - val_accuracy: 0.6800
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.9806 - accuracy: 0.66100/100 [==============================] - 0s 180us/sample - loss: 0.9123 - accuracy: 0.6600 - val_loss: 0.5721 - val_accuracy: 0.6800
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.6244 - accuracy: 0.68100/100 [==============================] - 0s 140us/sample - loss: 0.5804 - accuracy: 0.6800 - val_loss: 0.5535 - val_accuracy: 0.7000
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.2241 - accuracy: 0.34100/100 [==============================] - 0s 4ms/sample - loss: 1.2285 - accuracy: 0.3200 - val_loss: 1.2771 - val_accuracy: 0.3200
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.2660 - accuracy: 0.32100/100 [==============================] - 0s 768us/sample - loss: 1.0447 - accuracy: 0.4700 - val_loss: 0.6073 - val_accuracy: 0.7000
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.6862 - accuracy: 0.60100/100 [==============================] - 0s 140us/sample - loss: 0.7580 - accuracy: 0.5900 - val_loss: 0.7072 - val_accuracy: 0.6600
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.8443 - accuracy: 0.58100/100 [==============================] - 0s 728us/sample - loss: 0.6991 - accuracy: 0.6400 - val_loss: 0.4629 - val_accuracy: 0.7400
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1556 - accuracy: 0.38100/100 [==============================] - 0s 4ms/sample - loss: 1.1841 - accuracy: 0.3900 - val_loss: 0.7260 - val_accuracy: 0.7000
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.7652 - accuracy: 0.66100/100 [==============================] - 0s 140us/sample - loss: 0.7366 - accuracy: 0.6800 - val_loss: 0.7616 - val_accuracy: 0.6800
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.6522 - accuracy: 0.76100/100 [==============================] - 0s 160us/sample - loss: 0.7177 - accuracy: 0.6600 - val_loss: 0.4560 - val_accuracy: 0.7000
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.5475 - accuracy: 0.62100/100 [==============================] - 0s 170us/sample - loss: 0.5347 - accuracy: 0.6600 - val_loss: 0.5601 - val_accuracy: 0.7000
Epoch 5/20
 50/100 [==============>..........

Train on 100 samples, validate on 50 samples
Epoch 1/20
 50/100 [==============>...............] - ETA: 0s - loss: 1.1872 - accuracy: 0.40100/100 [==============================] - 0s 5ms/sample - loss: 1.2241 - accuracy: 0.3800 - val_loss: 0.9029 - val_accuracy: 0.4200
Epoch 2/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.8419 - accuracy: 0.46100/100 [==============================] - 0s 718us/sample - loss: 0.9080 - accuracy: 0.5400 - val_loss: 0.6373 - val_accuracy: 0.7000
Epoch 3/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.7125 - accuracy: 0.66100/100 [==============================] - 0s 130us/sample - loss: 0.6535 - accuracy: 0.8100 - val_loss: 0.6041 - val_accuracy: 0.6800
Epoch 4/20
 50/100 [==============>...............] - ETA: 0s - loss: 0.5757 - accuracy: 0.74100/100 [==============================] - 0s 170us/sample - loss: 0.6705 - accuracy: 0.6600 - val_loss: 0.4712 - val_accuracy: 0.7000
Epoch 5/20
 50/100 [==============>..........

INFO:tensorflow:Oracle triggered exit


#### Parece que con más epochs, saca mejor eval_metric en cada trial como esperaríamos

In [11]:
bayesian_opt_tuner.results_summary()

In [12]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_1_epoch',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             epochs=1,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.2798 - accuracy: 0.30100/100 [==============================] - 0s 4ms/sample - loss: 1.2411 - accuracy: 0.3400 - val_loss: 1.1852 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.7563 - accuracy: 0.36100/100 [==============================] - 0s 3ms/sample - loss: 1.6914 - accuracy: 0.3500 - val_loss: 1.6219 - val_accuracy: 0.3000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.4986 - accuracy: 0.0000e+100/100 [==============================] - 0s 3ms/sample - loss: 1.4723 - accuracy: 0.0100 - val_loss: 1.4128 - val_accuracy: 0.0200


Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.2104 - accuracy: 0.36100/100 [==============================] - 0s 4ms/sample - loss: 1.2332 - accuracy: 0.3400 - val_loss: 1.2968 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.6558 - accuracy: 0.28100/100 [==============================] - 0s 3ms/sample - loss: 1.5109 - accuracy: 0.3500 - val_loss: 1.5981 - val_accuracy: 0.3000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.3282 - accuracy: 0.26100/100 [==============================] - 0s 3ms/sample - loss: 1.2403 - accuracy: 0.3500 - val_loss: 1.2338 - val_accuracy: 0.3000


Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.3438 - accuracy: 0.32100/100 [==============================] - 0s 4ms/sample - loss: 1.5936 - accuracy: 0.2900 - val_loss: 1.0730 - val_accuracy: 0.7000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.1008 - accuracy: 0.68100/100 [==============================] - 0s 3ms/sample - loss: 1.2802 - accuracy: 0.5200 - val_loss: 0.8248 - val_accuracy: 0.6400
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.0868 - accuracy: 0.34100/100 [==============================] - 0s 3ms/sample - loss: 1.0501 - accuracy: 0.4700 - val_loss: 0.7083 - val_accuracy: 0.7000


Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.1727 - accuracy: 0.62100/100 [==============================] - 0s 4ms/sample - loss: 1.1413 - accuracy: 0.6600 - val_loss: 1.1242 - val_accuracy: 0.6800
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.3423 - accuracy: 0.34100/100 [==============================] - 0s 3ms/sample - loss: 1.3421 - accuracy: 0.3100 - val_loss: 1.2955 - val_accuracy: 0.4000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.1592 - accuracy: 0.26100/100 [==============================] - 0s 3ms/sample - loss: 1.1025 - accuracy: 0.3500 - val_loss: 1.1328 - val_accuracy: 0.3000


Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.2855 - accuracy: 0.30100/100 [==============================] - 0s 4ms/sample - loss: 1.5144 - accuracy: 0.4500 - val_loss: 1.1438 - val_accuracy: 0.7000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.2283 - accuracy: 0.30100/100 [==============================] - 0s 3ms/sample - loss: 1.4247 - accuracy: 0.2900 - val_loss: 1.0850 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.5415 - accuracy: 0.26100/100 [==============================] - 0s 3ms/sample - loss: 1.5051 - accuracy: 0.2700 - val_loss: 1.8710 - val_accuracy: 0.3200


INFO:tensorflow:Oracle triggered exit


In [13]:
bayesian_opt_tuner.results_summary()

### Y con solo un epoch saca resultados pobres de score, as expected

In [19]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_1_epoch_half_batchSize',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             epochs=1,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/2)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 0.9988 - accuracy: 0.70100/100 [==============================] - 0s 4ms/sample - loss: 0.9904 - accuracy: 0.6900 - val_loss: 0.9867 - val_accuracy: 0.7000
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 1.9732 - accuracy: 0.30100/100 [==============================] - 0s 3ms/sample - loss: 1.9310 - accuracy: 0.3100 - val_loss: 1.7779 - val_accuracy: 0.3800
Train on 100 samples, validate on 50 samples
 50/100 [==============>...............] - ETA: 0s - loss: 2.4052 - accuracy: 0.44100/100 [==============================] - 0s 3ms/sample - loss: 2.4249 - accuracy: 0.3400 - val_loss: 2.3642 - val_accuracy: 0.3200


INFO:tensorflow:Oracle triggered exit


In [22]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_1_epoch_aTenth_batchSize',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             epochs=1,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/10)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.2498 - accuracy: 0.50100/100 [==============================] - 0s 4ms/sample - loss: 1.2340 - accuracy: 0.3400 - val_loss: 1.1432 - val_accuracy: 0.3400
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.0871 - accuracy: 0.20100/100 [==============================] - 0s 3ms/sample - loss: 1.0342 - accuracy: 0.2100 - val_loss: 1.0344 - val_accuracy: 0.3000
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.0827 - accuracy: 0.40100/100 [==============================] - 0s 3ms/sample - loss: 1.1110 - accuracy: 0.3400 - val_loss: 1.1096 - val_accuracy: 0.3200


INFO:tensorflow:Oracle triggered exit


## VALIDAR CON HYPOTHESIS TESTING SI DISMINUIR EL BATCH_SIZE Y AUMENTAR EL EXECUTIONS_PER_TRIAL, TENIENDO FIJADO EL VALOR DE EPOCHS, MEJORA (EN VIRTUD EL P-VALOR) EL RESULTADO, EN VARIAS ITERACIONES 

In [23]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=30,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_1_epoch_aTenth_batchSize',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             epochs=1,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/10)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.4870 - accuracy: 0.20100/100 [==============================] - 0s 4ms/sample - loss: 1.1514 - accuracy: 0.3400 - val_loss: 1.0104 - val_accuracy: 0.3800
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.1129 - accuracy: 0.30100/100 [==============================] - 0s 3ms/sample - loss: 1.0559 - accuracy: 0.3500 - val_loss: 0.9665 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.3928 - accuracy: 0.30100/100 [==============================] - 0s 3ms/sample - loss: 1.1443 - accuracy: 0.3100 - val_loss: 1.0253 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.3048 - accuracy: 0.30100/100 [==============================] - 0s 4ms/sample - loss: 1.0353 - accuracy: 0.3600 - 

INFO:tensorflow:Oracle triggered exit


In [24]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=30,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_1_epoch_aTenth_batchSize',
    overwrite=True)


bayesian_opt_tuner.search(X_train, y_train,
             #epochs=1,
             validation_data=(X_test, y_test),
             batch_size=int(len(X_train)/10)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.5522 - accuracy: 0.20100/100 [==============================] - 0s 4ms/sample - loss: 1.3163 - accuracy: 0.3500 - val_loss: 1.3637 - val_accuracy: 0.3000
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 2.1120 - accuracy: 0.20100/100 [==============================] - 0s 4ms/sample - loss: 1.8913 - accuracy: 0.3100 - val_loss: 1.6870 - val_accuracy: 0.3800
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.2232 - accuracy: 0.30100/100 [==============================] - 0s 3ms/sample - loss: 1.1360 - accuracy: 0.2900 - val_loss: 1.1014 - val_accuracy: 0.3200
Train on 100 samples, validate on 50 samples
 10/100 [==>...........................] - ETA: 2s - loss: 1.2707 - accuracy: 0.40100/100 [==============================] - 0s 3ms/sample - loss: 1.3335 - accuracy: 0.3400 - 

INFO:tensorflow:Oracle triggered exit


###
RESUMEN PARAMS DE 'CONTROL': por cada trial (de los 'max_trials'), se ejecutan n 'executions_per_trial', donde en cada execution se lanzan un número de trainings/validations = epochs, con un dataset de entrenamiento de tamaño = batch_size   

## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 4


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.02353263 0.42498332 0.55148405]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.00119137 0.20332347 0.79548514]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))



random_searched_model_best_model_eval_set_acc: 0.8799999952316284
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [2]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [5]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test',
    overwrite=True)


In [6]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [7]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.1651 - accuracy: 0.0000e+100/100 [==============================] - 1s 8ms/sample - loss: 1.1012 - accuracy: 0.3200 - val_loss: 1.0219 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9637 - accuracy: 0.40100/100 [==============================] - 0s 190us/sample - loss: 0.9973 - accuracy: 0.3400 - val_loss: 0.9304 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9649 - accuracy: 0.25100/100 [==============================] - 0s 977us/sample - loss: 0.8970 - accuracy: 0.3600 - val_loss: 0.8110 - val_accuracy: 0.9600
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8181 - accuracy: 0.87100/100 [==============================] - 0s 189us/sample - loss: 0.7958 - accuracy: 0.9200 - val_loss: 0.7153 - val_accuracy: 0.7200
Epoch 5/5
 32/100 [========>.................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1095 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.0664 - accuracy: 0.3400 - val_loss: 1.0015 - val_accuracy: 0.4800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9614 - accuracy: 0.59100/100 [==============================] - 0s 938us/sample - loss: 0.9681 - accuracy: 0.4800 - val_loss: 0.9275 - val_accuracy: 0.5400
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9413 - accuracy: 0.50100/100 [==============================] - 0s 910us/sample - loss: 0.9014 - accuracy: 0.4600 - val_loss: 0.8519 - val_accuracy: 0.6600
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9045 - accuracy: 0.56100/100 [==============================] - 0s 1ms/sample - loss: 0.8379 - accuracy: 0.7700 - val_loss: 0.7861 - val_accuracy: 1.0000
Epoch 5/5
 32/100 [========>.....................] 

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2361 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.0029 - accuracy: 0.5900 - val_loss: 0.5997 - val_accuracy: 0.9600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.6390 - accuracy: 0.93100/100 [==============================] - 0s 977us/sample - loss: 0.6283 - accuracy: 0.7300 - val_loss: 0.4423 - val_accuracy: 1.0000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4679 - accuracy: 1.00100/100 [==============================] - 0s 180us/sample - loss: 0.4694 - accuracy: 0.9000 - val_loss: 0.3905 - val_accuracy: 0.8000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4635 - accuracy: 0.71100/100 [==============================] - 0s 180us/sample - loss: 0.4146 - accuracy: 0.7800 - val_loss: 0.2980 - val_accuracy: 0.9000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2214 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.1627 - accuracy: 0.3500 - val_loss: 1.1448 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1442 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 1.1256 - accuracy: 0.3500 - val_loss: 1.1127 - val_accuracy: 0.3000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1492 - accuracy: 0.25100/100 [==============================] - 0s 189us/sample - loss: 1.0971 - accuracy: 0.3500 - val_loss: 1.0856 - val_accuracy: 0.3000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0613 - accuracy: 0.37100/100 [==============================] - 0s 180us/sample - loss: 1.0698 - accuracy: 0.3500 - val_loss: 1.0621 - val_accuracy: 0.3000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2342 - accuracy: 0.0000e+100/100 [==============================] - 0s 4ms/sample - loss: 1.1901 - accuracy: 0.0500 - val_loss: 1.1290 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1202 - accuracy: 0.34100/100 [==============================] - 0s 180us/sample - loss: 1.0928 - accuracy: 0.3400 - val_loss: 1.0604 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0367 - accuracy: 0.34100/100 [==============================] - 0s 176us/sample - loss: 1.0284 - accuracy: 0.3400 - val_loss: 1.0053 - val_accuracy: 0.3200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0178 - accuracy: 0.28100/100 [==============================] - 0s 906us/sample - loss: 0.9792 - accuracy: 0.3400 - val_loss: 0.9504 - val_accuracy: 0.3400
Epoch 5/5
 32/100 [========>.................

INFO:tensorflow:Oracle triggered exit


In [8]:
random_search_tuner.results_summary()


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0333 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.0639 - accuracy: 0.3300 - val_loss: 1.3232 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.3889 - accuracy: 0.31100/100 [==============================] - 0s 918us/sample - loss: 1.1169 - accuracy: 0.4800 - val_loss: 0.5524 - val_accuracy: 0.6800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7179 - accuracy: 0.53100/100 [==============================] - 0s 918us/sample - loss: 0.5814 - accuracy: 0.7100 - val_loss: 0.4220 - val_accuracy: 0.7800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.3992 - accuracy: 0.81100/100 [==============================] - 0s 953us/sample - loss: 0.4776 - accuracy: 0.6700 - val_loss: 0.3719 - val_accuracy: 0.8400
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.4379 - accuracy: 0.25100/100 [==============================] - 0s 4ms/sample - loss: 1.2614 - accuracy: 0.3500 - val_loss: 1.1844 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0716 - accuracy: 0.40100/100 [==============================] - 0s 967us/sample - loss: 1.0703 - accuracy: 0.4000 - val_loss: 1.0554 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1345 - accuracy: 0.28100/100 [==============================] - 0s 209us/sample - loss: 1.0139 - accuracy: 0.3400 - val_loss: 0.9939 - val_accuracy: 0.3200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9741 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 0.9657 - accuracy: 0.3400 - val_loss: 0.9338 - val_accuracy: 0.3200
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0714 - accuracy: 0.34100/100 [==============================] - 0s 5ms/sample - loss: 1.0656 - accuracy: 0.3400 - val_loss: 1.0532 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0603 - accuracy: 0.31100/100 [==============================] - 0s 977us/sample - loss: 1.0478 - accuracy: 0.3400 - val_loss: 1.0403 - val_accuracy: 0.3400
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0634 - accuracy: 0.15100/100 [==============================] - 0s 1ms/sample - loss: 1.0339 - accuracy: 0.3700 - val_loss: 1.0283 - val_accuracy: 0.4400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0175 - accuracy: 0.43100/100 [==============================] - 0s 1ms/sample - loss: 1.0203 - accuracy: 0.4300 - val_loss: 1.0161 - val_accuracy: 0.5200
Epoch 5/5
 32/100 [========>.....................] - 

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 3.0992 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 2.4209 - accuracy: 0.3000 - val_loss: 1.3854 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2903 - accuracy: 0.46100/100 [==============================] - 0s 180us/sample - loss: 1.3110 - accuracy: 0.3400 - val_loss: 1.1153 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1134 - accuracy: 0.31100/100 [==============================] - 0s 928us/sample - loss: 1.0393 - accuracy: 0.3700 - val_loss: 1.0484 - val_accuracy: 0.4800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8468 - accuracy: 0.59100/100 [==============================] - 0s 947us/sample - loss: 0.9539 - accuracy: 0.5200 - val_loss: 0.8879 - val_accuracy: 0.5200
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0515 - accuracy: 0.71100/100 [==============================] - 0s 4ms/sample - loss: 0.9073 - accuracy: 0.6200 - val_loss: 0.6350 - val_accuracy: 0.6600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8147 - accuracy: 0.53100/100 [==============================] - 0s 947us/sample - loss: 0.5960 - accuracy: 0.6400 - val_loss: 0.4080 - val_accuracy: 0.8800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4616 - accuracy: 0.87100/100 [==============================] - 0s 938us/sample - loss: 0.4368 - accuracy: 0.8700 - val_loss: 0.3227 - val_accuracy: 0.9000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.3433 - accuracy: 0.96100/100 [==============================] - 0s 928us/sample - loss: 0.3910 - accuracy: 0.8900 - val_loss: 0.2579 - val_accuracy: 0.9800
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 4


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.08193266 0.42444137 0.49362594]]
single prediction of the bayes_opt_model_best_model on a test instance: [[3.1111806e-04 1.7690873e-01 8.2278013e-01]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 1.0
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam. pero con diferente número de max_trials (esto es, hyperparams combinations)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [17]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [19]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [20]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [21]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [22]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2436 - accuracy: 0.37100/100 [==============================] - 0s 4ms/sample - loss: 1.2129 - accuracy: 0.3400 - val_loss: 1.1148 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1118 - accuracy: 0.25100/100 [==============================] - 0s 209us/sample - loss: 1.0736 - accuracy: 0.3400 - val_loss: 1.0451 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0477 - accuracy: 0.31100/100 [==============================] - 0s 816us/sample - loss: 1.0087 - accuracy: 0.3400 - val_loss: 0.9411 - val_accuracy: 0.3400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0139 - accuracy: 0.25100/100 [==============================] - 0s 808us/sample - loss: 0.9235 - accuracy: 0.5000 - val_loss: 0.8475 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0708 - accuracy: 0.37100/100 [==============================] - 0s 4ms/sample - loss: 1.0759 - accuracy: 0.3500 - val_loss: 1.0748 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0515 - accuracy: 0.37100/100 [==============================] - 0s 199us/sample - loss: 1.0529 - accuracy: 0.3500 - val_loss: 1.0515 - val_accuracy: 0.3000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0557 - accuracy: 0.28100/100 [==============================] - 0s 199us/sample - loss: 1.0350 - accuracy: 0.3500 - val_loss: 1.0322 - val_accuracy: 0.3000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0167 - accuracy: 0.34100/100 [==============================] - 0s 783us/sample - loss: 1.0180 - accuracy: 0.3700 - val_loss: 1.0174 - val_accuracy: 0.4600
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2735 - accuracy: 0.21100/100 [==============================] - 0s 4ms/sample - loss: 1.2388 - accuracy: 0.2700 - val_loss: 1.0913 - val_accuracy: 0.4800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0578 - accuracy: 0.50100/100 [==============================] - 0s 807us/sample - loss: 0.9213 - accuracy: 0.5900 - val_loss: 0.6974 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8826 - accuracy: 0.59100/100 [==============================] - 0s 798us/sample - loss: 0.7277 - accuracy: 0.6500 - val_loss: 0.4899 - val_accuracy: 0.9200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.5222 - accuracy: 0.96100/100 [==============================] - 0s 199us/sample - loss: 0.5655 - accuracy: 0.7700 - val_loss: 0.5053 - val_accuracy: 0.6800
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0532 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.0413 - accuracy: 0.4800 - val_loss: 0.7923 - val_accuracy: 0.5000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8257 - accuracy: 0.43100/100 [==============================] - 0s 708us/sample - loss: 0.8347 - accuracy: 0.4000 - val_loss: 0.7403 - val_accuracy: 0.5800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7255 - accuracy: 0.53100/100 [==============================] - 0s 748us/sample - loss: 0.6745 - accuracy: 0.7400 - val_loss: 0.5836 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7289 - accuracy: 0.56100/100 [==============================] - 0s 189us/sample - loss: 0.6296 - accuracy: 0.6500 - val_loss: 0.5105 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9626 - accuracy: 0.59100/100 [==============================] - 0s 4ms/sample - loss: 1.0057 - accuracy: 0.3800 - val_loss: 0.9222 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9250 - accuracy: 0.31100/100 [==============================] - 0s 818us/sample - loss: 0.8975 - accuracy: 0.3500 - val_loss: 0.8417 - val_accuracy: 0.7800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8345 - accuracy: 0.68100/100 [==============================] - 0s 768us/sample - loss: 0.8318 - accuracy: 0.7900 - val_loss: 0.7777 - val_accuracy: 0.9800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7964 - accuracy: 0.90100/100 [==============================] - 0s 189us/sample - loss: 0.7817 - accuracy: 0.8800 - val_loss: 0.7178 - val_accuracy: 0.8600
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [23]:
random_search_tuner.results_summary()


In [24]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1159 - accuracy: 0.43100/100 [==============================] - 0s 4ms/sample - loss: 1.0793 - accuracy: 0.3400 - val_loss: 0.9589 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9677 - accuracy: 0.25100/100 [==============================] - 0s 798us/sample - loss: 0.9173 - accuracy: 0.5700 - val_loss: 0.8378 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8429 - accuracy: 0.71100/100 [==============================] - 0s 783us/sample - loss: 0.8401 - accuracy: 0.6500 - val_loss: 0.7593 - val_accuracy: 0.7200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8112 - accuracy: 0.62100/100 [==============================] - 0s 808us/sample - loss: 0.7739 - accuracy: 0.8400 - val_loss: 0.7037 - val_accuracy: 0.9200
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [25]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [26]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 0


## Get model weights:

In [27]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [28]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.00113969 0.35658044 0.64227986]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.07741494 0.41732666 0.5052584 ]]


## Predictions on the test set:

In [29]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [30]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.9800000190734863
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam. pero con diferente número de max_trials (esto es, hyperparams combinations)

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [2]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [5]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [6]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [7]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.0394 - accuracy: 0.65100/100 [==============================] - 1s 8ms/sample - loss: 1.0321 - accuracy: 0.4100 - val_loss: 0.9536 - val_accuracy: 0.5600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9534 - accuracy: 0.59100/100 [==============================] - 0s 983us/sample - loss: 0.9305 - accuracy: 0.6200 - val_loss: 0.8507 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8466 - accuracy: 0.75100/100 [==============================] - 0s 199us/sample - loss: 0.8618 - accuracy: 0.6500 - val_loss: 0.7816 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8444 - accuracy: 0.59100/100 [==============================] - 0s 189us/sample - loss: 0.8023 - accuracy: 0.6500 - val_loss: 0.7379 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [8]:
random_search_tuner.results_summary()


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1780 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.1210 - accuracy: 0.5100 - val_loss: 0.9449 - val_accuracy: 0.6800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9724 - accuracy: 0.65100/100 [==============================] - 0s 194us/sample - loss: 0.9235 - accuracy: 0.4700 - val_loss: 0.9066 - val_accuracy: 0.4200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9118 - accuracy: 0.40100/100 [==============================] - 0s 199us/sample - loss: 0.8723 - accuracy: 0.4300 - val_loss: 0.8182 - val_accuracy: 0.4400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8013 - accuracy: 0.43100/100 [==============================] - 0s 1ms/sample - loss: 0.7959 - accuracy: 0.5500 - val_loss: 0.7184 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................] 

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 0
number of bayesian optimized models: 0


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.15195647 0.3824394  0.4656042 ]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.08514515 0.36450717 0.5503476 ]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.699999988079071
bayes_opt_model_best_model_eval_set_acc: 0.9200000166893005


In [1]:
### Al lanzar con un solo max_trial, vemos que coincide nuestro 'CategoricalAccuracy' con el 'score' informado en el summary

In [2]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open("model.json", "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights("model.h5")
print("Saved model to disk")




NameError: name 'bayes_opt_model_best_model' is not defined

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [4]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [6]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [7]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [8]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [9]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.1483 - accuracy: 0.0000e+100/100 [==============================] - 1s 8ms/sample - loss: 1.0586 - accuracy: 0.3100 - val_loss: 0.7178 - val_accuracy: 0.6600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7164 - accuracy: 0.71100/100 [==============================] - 0s 768us/sample - loss: 0.7975 - accuracy: 0.6300 - val_loss: 0.5403 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.5618 - accuracy: 0.68100/100 [==============================] - 0s 180us/sample - loss: 0.7734 - accuracy: 0.6500 - val_loss: 0.5402 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7486 - accuracy: 0.46100/100 [==============================] - 0s 170us/sample - loss: 0.6169 - accuracy: 0.6100 - val_loss: 0.4960 - val_accuracy: 0.6800
Epoch 5/5
 32/100 [========>.................

INFO:tensorflow:Oracle triggered exit


In [10]:
random_search_tuner.results_summary()


In [11]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.3701 - accuracy: 0.25100/100 [==============================] - 0s 4ms/sample - loss: 1.2249 - accuracy: 0.3400 - val_loss: 1.2083 - val_accuracy: 0.3400
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2240 - accuracy: 0.31100/100 [==============================] - 0s 763us/sample - loss: 1.1488 - accuracy: 0.4600 - val_loss: 1.1662 - val_accuracy: 0.5000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2085 - accuracy: 0.37100/100 [==============================] - 0s 189us/sample - loss: 1.0977 - accuracy: 0.5300 - val_loss: 1.0948 - val_accuracy: 0.4800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0650 - accuracy: 0.56100/100 [==============================] - 0s 249us/sample - loss: 1.0422 - accuracy: 0.5300 - val_loss: 1.0411 - val_accuracy: 0.4800
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [12]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [13]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 0
number of bayesian optimized models: 0


## Get model weights:

In [14]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)


 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [15]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.0168224  0.1594499  0.82372767]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.13170053 0.43555695 0.43274254]]


## Predictions on the test set:

In [16]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [17]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.699999988079071
bayes_opt_model_best_model_eval_set_acc: 0.5199999809265137


In [18]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open("model.json", "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights("model.h5")
print("Saved model to disk")



Saved model to disk


In [19]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory_model_arch_json=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2/model.json')
directory_model_h5=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2/model.h5')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open(directory_model_arch_json, "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights(directory_model_h5)
print("Saved model to disk")



Saved model to disk


In [21]:
# Load model:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open(directory_model_arch_json, 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(directory_model_h5)

In [22]:
print("Loaded model from disk")
# evaluate loaded model on test data
'''
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
'''
loaded_model.predict(X_test)



Loaded model from disk


array([[0.18691356, 0.40708265, 0.4060038 ],
       [0.28102165, 0.500248  , 0.21873033],
       [0.12444609, 0.3484776 , 0.5270763 ],
       [0.1869745 , 0.43074673, 0.38227883],
       [0.19077368, 0.4244052 , 0.3848211 ],
       [0.29011714, 0.49235243, 0.21753041],
       [0.22125033, 0.45003098, 0.32871872],
       [0.16334489, 0.46057817, 0.37607694],
       [0.19597395, 0.40819627, 0.3958298 ],
       [0.2158887 , 0.43205994, 0.35205135],
       [0.16219898, 0.44247136, 0.39532968],
       [0.3044084 , 0.45945838, 0.23613323],
       [0.29613838, 0.49743438, 0.20642725],
       [0.30076176, 0.4607826 , 0.23845556],
       [0.27742857, 0.49995244, 0.222619  ],
       [0.17926702, 0.44602126, 0.37471175],
       [0.14081255, 0.39579558, 0.46339187],
       [0.21615186, 0.4175978 , 0.36625034],
       [0.18703076, 0.41160712, 0.40136215],
       [0.14627685, 0.39922458, 0.4544986 ],
       [0.2903147 , 0.46196306, 0.24772227],
       [0.16836071, 0.42768717, 0.40395215],
       [0.

In [23]:
print("Loaded model from disk")
# evaluate loaded model on test data
'''
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
'''
#loaded_model.predict(X_test)
test_set_bayes_opt_loaded_model_predictions = loaded_model.predict(X_test)
categ_acc_bayes_opt_loaded_model_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_loaded_model_test_set.update_state(y_test, test_set_bayes_opt_loaded_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_loaded_model_test_set.result().numpy()))



Loaded model from disk
bayes_opt_model_best_model_eval_set_acc: 0.5199999809265137
